In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import time
from datetime import datetime

In [2]:
path = r"C:\Users\Mathiass\OneDrive - Universität Zürich UZH\Documents\mt_literature\data"

In [3]:
s_time = time.time()
data = pd.read_parquet(path+"\sp500_op_ret.parquet")
e_time = time.time()

In [4]:
print("Read without chunks: ", (e_time-s_time), "seconds")

Read without chunks:  1.3819060325622559 seconds


In [21]:
time_dict = {}

In [26]:
a = datetime.now()

In [27]:
b = datetime.now() - a

In [28]:
t = b.total_seconds()

In [29]:
t

3.162222

In [32]:
round(divmod(t, 60)[1], 2)

3.16

In [17]:
dayss = 24 * 60 * 60

In [18]:
days, rem = divmod(t, dayss)

In [20]:
divmod(rem, 60)


(3, 51)

In [14]:
hourss = 60 * 60

In [15]:
days

1

In [16]:
hours, secs = divmod(rem, hourss)

In [17]:
hours

1

In [18]:
secs

0

In [19]:
time_dict["days"] = 1

In [20]:
time_dict["hours"] = 2

In [21]:
time_dict["secs"] = 3

In [22]:
time_dict

{'days': 1, 'hours': 2, 'secs': 3}

In [23]:
total_dict = {}

In [24]:
total_dict["loop1"] = time_dict

In [25]:
total_dict

{'loop1': {'days': 1, 'hours': 2, 'secs': 3}}

In [26]:
pd.DataFrame.from_dict(total_dict).T

,days,hours,secs
loop1,1,2,3


In [ ]:
data.describe()

In [ ]:
data

In [ ]:
len(data["secid"].unique())

In [ ]:
data3 = data.iloc[:, :2]

In [ ]:
data3.groupby("date").nunique().describe()

In [ ]:
data3["date"].nunique()

In [ ]:
list(data3["secid"].unique())

In [ ]:
data[data["secid"] == 101375]

In [ ]:

data3[data3["secid"] == 101375].nunique()["date"]

In [ ]:
# how many datapoints per time does each stock have?
ts_per_secid = []
unique_secid = data3["secid"].unique()

for i in unique_secid:
    unique_dates = data3[data3["secid"] == i].nunique()["date"]
    ts_per_secid.append((i, unique_dates))
    

In [ ]:
# number of stocks that have at least one entry in each date
df = pd.DataFrame(ts_per_secid)
len(df[df[1] == 311])

In [ ]:
list1 = list(df[0])

In [ ]:
# save number of stocks that have at least one entry in each date
secid_complete = df[df[1] == 311][0]

In [ ]:
# nr of unique stocks
len(list1)

In [ ]:
df

In [ ]:
secid_complete

In [ ]:
data[data["secid"].isin(secid_complete)].head(40).reset_index()

In [ ]:
data[data["secid"].isin(secid_complete)].tail(20).reset_index()

In [ ]:
data_complete = data[data["secid"].isin(secid_complete)].copy()

In [ ]:
# Set Call to 1, Put to 0
data_complete.loc[data_complete["cp_flag"] == "C", "cp_flag"] = 1
data_complete.loc[data_complete["cp_flag"] == "P", "cp_flag"] = 0
# Convert object type to int or date
data_complete["cp_flag"] = data_complete["cp_flag"].astype(int)
data_complete["date"] = pd.to_datetime(data_complete["date"])
data_complete["exdate"] = pd.to_datetime(data_complete["exdate"])

In [ ]:
data_complete.dtypes

In [ ]:
data_complete

In [ ]:
data_complete.groupby(["secid", "date"]).agg("mean")

In [ ]:
data_complete = data_complete.groupby(["secid", "date"]).agg("mean").sort_values("date").reset_index()

In [ ]:
data_complete

In [ ]:
(data_complete.groupby("date").agg("count") == 33).all().all()

In [ ]:
data_complete

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
10263 / 33

In [ ]:
data_complete = data_complete.groupby("date").agg("mean")

In [ ]:
# Data averaged over stocks and date (311 rows) -> for Time Series prediction
y = data_complete["option_ret"]
X = data_complete.drop(["option_ret", "secid", "optionid", "days_no_trading"], axis=1)

In [ ]:
from pandas.tseries.offsets import MonthEnd

In [ ]:
# All data (3825531 rows) -> for Cross-sectional analysis
# Set Call to 1, Put to 0
# data.loc[data["cp_flag"] == "C", "cp_flag"] = 1
# data.loc[data["cp_flag"] == "P", "cp_flag"] = 0
# Convert object type to int or date
# data["cp_flag"] = data["cp_flag"].astype(int)
data["date"] = pd.to_datetime(data["date"])
data["exdate"] = pd.to_datetime(data["exdate"])
# bring dates to end of month dates (in order to correctly merge with other datasets)
data["date"] = data["date"] + MonthEnd(0)
# split off y
data = data.drop(data[data["option_ret"].isnull()].index) # remove 138 NaN option returns
data = data.reset_index(drop=True)
y = data["option_ret"]
X = data.drop(["option_ret", "secid", "optionid", "days_no_trading", "date", "exdate"], axis=1)


In [ ]:
# # only use certain features?
# X = X[["cp_flag", "impl_volatility", "delta"]]

In [ ]:
# Load csv data
s_time = time.time()
mapping_table = pd.read_csv(path+"\mapping_table.csv")
e_time = time.time()
print("Read without chunks: ", (e_time-s_time), "seconds")

In [ ]:
mapping_table

In [ ]:
df = data.merge(mapping_table, on="secid")

In [ ]:
df

In [ ]:
# correct format important here, otherwise will see month as day and vice versa
df["sdate"] = pd.to_datetime(df["sdate"], format = "%d/%m/%Y %H:%M")
df["edate"] = pd.to_datetime(df["edate"], format = "%d/%m/%Y %H:%M")

In [ ]:
df_filter = df[(df["date"] >= df["sdate"]) & ((df["date"] <= df["edate"]) | (df["edate"] >= "2020-12-31"))]

In [ ]:
df_filter

In [ ]:
3818687

In [ ]:
data

In [ ]:
len(df_filter)

In [ ]:
len(data)

In [ ]:
# should have 3825393 rows (4 more than below)
data

In [ ]:
# should have 3825389 rows
df_filter

In [ ]:
a = set(data[["optionid", "date"]].itertuples(index=False, name=None))

In [ ]:
b = set(df_filter[["optionid", "date"]].itertuples(index=False, name=None))

In [ ]:
c = list(a - b)

In [ ]:
len(c)

In [ ]:
e = list(data["optionid"])

In [ ]:
f = list(df_filter["optionid"])

In [ ]:
d = list(set(e) - set(f))

In [ ]:
u = set([x[0] for x in c])

In [ ]:
z = set([x[0] for x in c]) - set(d)

In [ ]:
z

In [ ]:
len(d)

In [ ]:
data[data["optionid"].isin(u)][data[data["optionid"].isin(u)]["secid"] == 102362                   ]

In [ ]:
df_filter[df_filter["optionid"].isin(u)][df_filter[df_filter["optionid"].isin(u)]["secid"] == 102362 ]

In [ ]:
df_filter["edate"]

In [ ]:
# 4 observations are dropped because permno is ambiguous in the 94 char. feature list (Moodys spin off)
data[data["optionid"].isin(u)]["secid"].value_counts()

In [ ]:
data[data["optionid"].isin(d)]

In [ ]:
# Load PARQUET data (3sec instead of 38sec -> speedup of 10x)
s_time = time.time()
data2 = pd.read_parquet(path+"\Gu2020_datashare\datashare.parquet")
e_time = time.time()
print("Read without chunks: ", (e_time-s_time), "seconds")

In [ ]:
from pandas.tseries.offsets import MonthEnd

In [ ]:
# Lag features by -1 to align with option features
data2["DATE"] = pd.to_datetime(data2["DATE"].astype(str), format='%Y%m%d') - MonthEnd(1)

In [ ]:
data2

In [ ]:
data2[data2["permno"] == 48506]

In [ ]:
(data2.loc[data2["permno"] == 93429, "sic2"])

In [ ]:
# Manually insert SIC codes
data2.loc[data2["permno"] == 18312, "sic2"] = 28 #Moderna https://sec.report/CIK/0001682852
data2.loc[data2["permno"] == 18428, "sic2"] = 28 #Dow Inc https://sec.report/Ticker/DOW
data2.loc[data2["permno"] == 93429, "sic2"] = 62 #CBOE Global Markets https://sec.report/Ticker/CBOE
data2.loc[data2["permno"] == 18143, "sic2"] = 28 #Linde https://sec.report/CIK/0001707925
data2.loc[data2["permno"] == 19285, "sic2"] = 35 #Carrier Global Corp https://sec.report/Ticker/CARR
data2.loc[data2["permno"] == 18592, "sic2"] = 1 #Corteva https://sec.report/CIK/0001755672
data2.loc[data2["permno"] == 17942, "sic2"] = 20 #Keurig Dr Pepper https://sec.report/Ticker/kdp
data2.loc[data2["permno"] == 18420, "sic2"] = 48 #FOX Corp https://sec.report/Ticker/foxa
data2.loc[data2["permno"] == 20057, "sic2"] = 28 #Viatris https://sec.report/Ticker/foxa
data2.loc[data2["permno"] == 15850, "sic2"] = 72 #Match Group https://sec.report/Ticker/mtch
data2.loc[data2["permno"] == 32791, "sic2"] = 35 #Weatherford https://sec.report/Ticker/wft
data2.loc[data2["permno"] == 19286, "sic2"] = 36 #OTIS https://sec.report/Ticker/OTIS
data2.loc[data2["permno"] == 17700, "sic2"] = 73 #Ceridian https://sec.report/Ticker/CDAY
data2.loc[data2["permno"] == 18724, "sic2"] = 39 #Amcor https://sec.report/Ticker/AMCR
data2.loc[data2["permno"] == 38850, "sic2"] = 24 #Skyline https://sec.report/Ticker/sky
data2.loc[data2["permno"] == 19807, "sic2"] = 38 #Vontier https://sec.report/Ticker/VNT
data2.loc[data2["permno"] == 78840, "sic2"] = 48 #IAC Interactive Corp ->sp500 const. list
# ----
data2.loc[data2["permno"] == 81594, "sic2"] = 13 #Dynegy, only 8 were NaN, rest was =13
data2.loc[data2["permno"] == 88601, "sic2"] = 49 #Mirant, first 4 were NaN, rest =49
data2.loc[data2["permno"] == 80913, "sic2"] = 73 #ACS, last month NaN, rest =73
data2.loc[data2["permno"] == 44652, "sic2"] = 54 #American Stores Company, last month NaN, rest =54
data2.loc[data2["permno"] == 76563, "sic2"] = 54 #Meyer Fred Inc., last month NaN, rest =54




In [ ]:
data2

In [ ]:
# Our data starts at 1996-01-31
df = data2[data2["DATE"] > "1995-12-31"]
df = df.reset_index()
df = df.rename(columns={"DATE": "date"})

In [ ]:
# unique set of dates to loop over and take mean/median of all stocks in that month
date_unique = list(df["date"].unique())

In [ ]:
# FILL MISSING VALUES. COMMENT THIS OUT FOR ORIGINAL DF_FINAL.PARQUET. loop over each date and fill NA
from tqdm import tqdm
for i in tqdm(date_unique):
    df[df["date"] == i] = df[df["date"] == i].fillna(df[df["date"] == i].mean())

In [ ]:
df.loc[6419, :]

In [ ]:
# % missing values in stock characteristics?
np.sum(np.sum(df.isnull())) / (df.shape[0] * df.shape[1])

In [ ]:
df[df["date"] == date_unique[0]].mean(axis=0, numeric_only=True)

In [ ]:
df[df["date"] == date_unique[0]]["beta"][655]

In [ ]:
# Check what should be inserted by for loop
df[df["date"] == date_unique[0]].fillna(df[df["date"] == date_unique[0]].mean(axis=0, numeric_only=False))["beta"][655]

In [ ]:
# Check what should be inserted by for loop
df[df["date"] == date_unique[0]]["std_turn"].mean()

In [ ]:
0.9909510930939289

In [ ]:
df["permno"].nunique()

In [ ]:
df_filter

In [ ]:
# final dataset "INNER" -> REMOVE VALUES THAT HAVE NO 94 CHARACTERISTICS, "left" -> how to impute?
final_df = df_filter.merge(df, on=["date", "permno"], how="inner")

In [ ]:
final_df["permno"].nunique()

In [ ]:
final_df["secid"].nunique()

In [ ]:
final_df[final_df["permno"] == 56266]

In [ ]:
final_df

In [ ]:
permno_unique = list(final_df["permno"].unique())

In [ ]:
permno_unique

In [ ]:
final_df.loc[final_df["permno"] == 56266, "chatoia"]

In [ ]:
np.sum(final_df.loc[final_df["permno"] == 56266, :].isnull()).sort_values()[-30:]

In [ ]:
# Put option returns at the end
opt_ret = final_df.pop("option_ret")
final_df["option_ret"] = opt_ret

In [ ]:
# final_df.to_csv("final")

In [ ]:
# -> How to save this table so that one can efficiently get it back? feather? pickle?

In [ ]:
df_filter

In [ ]:
final_df

In [ ]:
# how many rows got remove from the inner joing
len(df_filter) - len(final_df)

In [ ]:
# How many "sic2" NaNs per "secid"
final_df[final_df["sic2"].isnull()]["secid"].value_counts()

In [ ]:
# SAVE FINAL DF TO PARQUET

# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.compose import make_column_transformer

# onehotencoder = make_column_transformer((OneHotEncoder(), ["cp_flag", "sic2"]), remainder="drop")
# transformed = onehotencoder.fit_transform(final_df)
# transformed_df = pd.DataFrame(transformed.toarray(), columns=onehotencoder.transformers_[0][1].get_feature_names_out(["cp_flag", "sic2"]))

# # drop old columns cp_flag, sic2
# final_df = final_df.drop(["cp_flag", "sic2"], axis=1)

# # concatenate both dataframes (memory intensive?)
# final_df = pd.concat([final_df, transformed_df], axis=1)

# # sort values by date! (secondly by secid -> but better to shuffle within date later)
# final_df = final_df.sort_values(["date", "secid"]).reset_index(drop=True)

# # remove useless columns for dataanalsis
# final_df = final_df.drop(["secid", "optionid", "exdate", "sdate", "edate", "permno", "index"], axis=1)

# # to parquet (no r -> doesnt work somehow, \f gets transformed to \x0c)
# final_df.to_parquet(fr"{path}\final_df_filledmean.parquet")

In [ ]:
# Load final file
final_df = pd.read_parquet(path+"final_df_filledmean.parquet")

In [ ]:
final_df["option_ret"]

In [ ]:
121 + 65

In [ ]:
# sic2 = final_df["sic2"]

In [ ]:
# sic2.nunique()

In [ ]:
# final_df[["sic2"]]

In [ ]:
# encoder_df = pd.DataFrame(encoder.fit_transform(final_df[["sic2"]]))

In [ ]:
# final_df

In [ ]:
data

In [ ]:
final_df

In [ ]:
3823386

### Idea:

- classification vs. regression. -> use of SVM/SVR as additional models
- classif: prediction of up/neutral/down (+1/0/-1) ?
- add sentiment columns to each option? -> sentiment analysis of past stock price history

##### Notes
- Get more Data?: Generate samples with generative models? (diffusion models)?
- Fully Convolutional / Convolutional Nets for varying input size per point in time? or for additional feature for each row
- Use underlying stock/ option time series as additional column features? But what about missing history?